<a href="https://colab.research.google.com/github/chetan-ade/FYP/blob/master/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install requests
!pip install wwo_hist
!pip install datetime
!pip install numpy
!pip install scipy
!pip install matplotlib
!pip install pompy

  Created wheel for wwo-hist: filename=wwo_hist-0.0.4-cp36-none-any.whl size=4216 sha256=cbd636d2f5a4999582ce87de01bee38821bac895898f2c3eee04d05e2d49ab81
  Stored in directory: /root/.cache/pip/wheels/c5/0e/16/329d9233f3b0b7e5fe81b09c5519a193727999c6ae77577ca9
Successfully built wwo-hist
     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 235kB 8.5MB/s 
  Created wheel for pompy: filename=pompy-0.1.1-cp36-none-any.whl size=17006 sha256=56c536197ac2abea1e5996ca27d0d80be98c3793e23202243a18380148171048
  Stored in directory: /root/.cache/pip/wheels/de/af/e1/8d78a0942c286ffc595d4232c22dbdb06ec868e3c131ad8c8c
Successfully built pompy


In [0]:
import os
import csv
import requests
import datetime
from datetime import date
from wwo_hist import retrieve_hist_data

#PATH TO SAVE CSV FILE
os.chdir("/content")

api_key = '91a5c7d65b78493086651634202801'
start_date = '01-JUL-2008'
end_date = date.today().strftime("%d-%b-%Y").upper()
frequency= 24

url = 'http://api.worldweatheronline.com/premium/v1/weather.ashx?'

params = {
    'key': api_key,
    'num_of_days': '1',
    'tp': '24',
    'cc': 'no',
    'mca': 'no',
    'format': 'json',
    'includelocation': 'yes',
    'q': None,
    'date': None,
    }

def getHistoricalData(location):
    hist_weather_data = retrieve_hist_data(api_key, [location], start_date, end_date, frequency, location_label = False, export_csv = True, store_df = False)

def getPredictedData(location):
    apiList = []
    params['q'] = location
    for i in range(16):
        curDate = (datetime.datetime.today() + datetime.timedelta(days=i)).strftime('%Y-%m-%d')
        params['date'] = curDate
        r = requests.get(url = url, params = params)
        dataJson = r.json()
        location = dataJson['data']['nearest_area'][0]['areaName'][0]['value']
        curDir = dataJson['data']['weather'][0]['hourly'][0]['winddirDegree']
        curSpd = dataJson['data']['weather'][0]['hourly'][0]['windspeedKmph']
        element = {'Date':curDate,'Direction': curDir, 'Speed': curSpd}
        apiList.append(element)
    print(location)
    print(apiList)
    return (apiList,location)

def listToCSV(dataList,csv_file):
    csv_columns = ['Date','Direction','Speed']
    # csv_file = "PredictedFromAPI.csv"
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dataList:
                writer.writerow(data)
    except IOError:
        print("Error while creating",csv_file)

def createWindList(apiList,modelList):
    windList = []
    for i in range(len(apiList)):
        Date = apiList[i]['Date']
        Direction = (int(apiList[i]['Direction']) + int(modelList[i]['Direction']))/2
        Speed = (int(apiList[i]['Speed']) + int(modelList[i]['Speed']))/2
        windList.append({'Date':Date,'Direction':str(Direction),'Speed':str(Speed)})
    print(windList)



In [0]:
apiList,location = getPredictedData('19.0368,73.0158')

Belapurpada
[{'Date': '2020-03-21', 'Direction': '200', 'Speed': '8'}, {'Date': '2020-03-22', 'Direction': '192', 'Speed': '10'}, {'Date': '2020-03-23', 'Direction': '185', 'Speed': '9'}, {'Date': '2020-03-24', 'Direction': '213', 'Speed': '9'}, {'Date': '2020-03-25', 'Direction': '189', 'Speed': '12'}, {'Date': '2020-03-26', 'Direction': '194', 'Speed': '8'}, {'Date': '2020-03-27', 'Direction': '218', 'Speed': '10'}, {'Date': '2020-03-28', 'Direction': '144', 'Speed': '9'}, {'Date': '2020-03-29', 'Direction': '180', 'Speed': '8'}, {'Date': '2020-03-30', 'Direction': '187', 'Speed': '9'}, {'Date': '2020-03-31', 'Direction': '246', 'Speed': '10'}, {'Date': '2020-04-01', 'Direction': '179', 'Speed': '10'}, {'Date': '2020-04-02', 'Direction': '255', 'Speed': '12'}, {'Date': '2020-04-03', 'Direction': '237', 'Speed': '12'}, {'Date': '2020-04-04', 'Direction': '199', 'Speed': '8'}, {'Date': '2020-04-05', 'Direction': '241', 'Speed': '9'}]


In [0]:
getHistoricalData(location)

# Useful Information

Data is being collected from various sources as listed below:

1.   World Weather Online
     <br>Link: https://www.worldweatheronline.com/developer/
     <br>Supporting Package: https://pypi.org/project/wwo-hist/ 
    
     World Weather Online’s allows users to receive weather data from 1st July 2008 until present day. 
     <br>A supporting package <i>wwo-hist</i> is used to retrieve and transform historical weather data from World Weather Online into csv file.
     <br>wwo-hist is not installed by default and is installed by :
     > !pip install wwo-hist
     
     An API Key is required to retrieve data from World Weather Online. 
     <br>Free trial API is used. 
     <br>It lasts for 60 days, allows 500 requests per day and requires no credit card.
     > API KEY = 91a5c7d65b78493086651634202801
     <br>Key generated on : 28 Jan 2020
     
     From wwo-hist, we use the <i>retrieve_hist_data()</i> function which has the following parameters :
     
     <i>api_key: string</i>
     <br>(Premium/ free trial) API key from worldweatheronline.com

     <i>location_list: list of string</i>
     <br>S Zipcode, UK Postcode, Canada Postalcode, IP address, Latitude/Longitude (decimal degree) or city name

     <i>start_date: string</i>
     <br>Preferred date format: 'dd-mmm-yyyy'

     <i>end_date: string</i>
     <br>Preferred date format: 'dd-mmm-yyyy'

     <i>frequency: integer</i>
     <br>1, 3, 6, 12, 24
     <br>1 hourly, 3 hourly, 6 hourly, 12 hourly (day/night) or 24 hourly (day average)

     <i>location_label: bool, default = False</i>
     <br>If True, all column names will have city name as prefix.

     <i>export_csv: bool, default = True</i>
     <br>If False, no csv file will be exported to current directory.

     <i>store_df: bool, default = False</i>
     <br>If True, retrieved dataframe(s) will be stored as list in the work space.
     
     * Drawback- City name must be a single word. Weather can be retrieved using Latitude OR Longitude, NOT BOTH.
2.   Local Weather API
     <br>Link: https://www.worldweatheronline.com/developer/api/docs/local-city-town-weather-api.aspx

     This API is a service provided by World Weather Online.<br>It is used to retrieve weather data for the next 15 days.<br>The acquired data is then used in order to further increase the accuracy of prediction.